In [1]:
using CPUTime
using DelimitedFiles
using Base
using DataStructures

In [2]:
numVertex=4
Edges = [(4,1,2), (1,2,1), (2,3,1), (3,4,1), (4,2,3), (1,3,5)]
AdjList = [Vector{}() for _ in 1:numVertex]
for ln in Edges
    push!(AdjList[ln[1]], (ln[1], ln[2],ln[3]))
    push!(AdjList[ln[2]], (ln[2], ln[1],ln[3]))
end

In [3]:
# for in_n_10e5_m_5e6.in, generate the file from graph_generator.ipynb
# function readDataFromFile(filename="in_n_10e5_m_5e6.in")
function readDataFromFile(filename="in7.txt")
    open(filename, "r") do f 
        n, m = split(readline(f))
        n = parse(Int, n)
        AdjList = [Vector{}() for _ in 1:n]
#         println(n," ",m)
        for ln in eachline(f)
#             a,b,c = split(ln)
            a,b,c = readdlm(IOBuffer(ln), Int)
            if(a==0)
                a=n
            end
            if(b==0)
                b=n
            end
            push!(AdjList[a], (a,b,c))
            push!(AdjList[b], (b,a,c))
#             println(a," ",b," ",c)
        end
        return n, AdjList
    end
end
numVertex, AdjList = readDataFromFile();

In [4]:
println(numVertex)

10000


In [5]:
uf = IntDisjointSets(numVertex+5);


In [6]:
MAX_INT=2000000000;
minCost = fill(MAX_INT, numVertex+1);
minNodeTgt = Vector{Int}(1:numVertex+1);
minNodeSrc = Vector{Int}(1:numVertex+1);

In [7]:
res = 0 
mst = Vector{}()

0-element Array{Any,1}

In [8]:
function initMinCostArray()
    for i = 1:numVertex
        minCost[i] = MAX_INT
        minNodeTgt[i] = i
        minNodeSrc[i] = i
    end
end

initMinCostArray (generic function with 1 method)

In [9]:
function findMinCostVertex()
    for i in 1:length(AdjList)
        for j in AdjList[i]
#             println(i," -> ", j[1], "=", j[2], " ",  !in_same_set(uf, i, j[1])  )
            set1 = find_root(uf, j[1])
            set2 = find_root(uf, j[2])
            if set1 != set2
                if(minCost[set1] > j[3] )
                    minCost[set1] = j[3]
                    minNodeTgt[set1] = j[2]
                    minNodeSrc[set1] = j[1]
                end
            end
        end
    end
end

findMinCostVertex (generic function with 1 method)

In [10]:
function contractVertexAndMergeAdjList()
    for i in 1:numVertex
        if(minCost[i]!= MAX_INT && !in_same_set(uf, minNodeSrc[i], minNodeTgt[i]))
            # Connect the vertices, add mst to answer
            set1 = find_root(uf, minNodeSrc[i])
            set2 = find_root(uf, minNodeTgt[i])
            union!(uf, minNodeSrc[i], minNodeTgt[i])
            global res+=minCost[i]
            push!(mst, (minNodeSrc[i], minNodeTgt[i], minCost[i]))
            # Merge AdjList 
            len1 = length(AdjList[set1])
            len2 = length(AdjList[set2])
            if (len1> len2)
                for j in AdjList[set2]
                    push!(AdjList[set1],j)
                end
                AdjList[set2] = Vector{}()
            else
                for j in AdjList[set1]
                    push!(AdjList[set2],j)
                    AdjList[set1] = Vector{}()
                end
            end
        end
    end
end

contractVertexAndMergeAdjList (generic function with 1 method)

In [11]:
function boruvka_MST(maxItr = convert(Int64, round(log2(numVertex)+1, digits=0)))
    i=1
    println("Max iteration: ", maxItr)
    while(i<maxItr && length(mst)< numVertex-1)
        i+=1
        println(i, " ", length(mst))
        initMinCostArray()
        findMinCostVertex()
        contractVertexAndMergeAdjList()
        # println(res)
        # println(length(mst))
    end
end

boruvka_MST (generic function with 2 methods)

In [12]:
@time @CPUtime boruvka_MST()

Max iteration: 14
2 0
3 7491
4 9477
5 9919
6 9986
7 9998
elapsed CPU time: 1.899149 seconds
  1.978484 seconds (9.29 M allocations: 226.088 MiB, 2.06% gc time)


In [13]:
println(length(mst))

9999


In [14]:
println(res)

6064035
